### Imports

In [1]:
from concurrent.futures import ThreadPoolExecutor, TimeoutError
from hfsql_guide.settings.credentials import dsn, user, passwd
from hfsql_guide.settings.paths import DATA_DIR, PARQUET_DIR, BLACKLISTED_TABLES, FAILED_TABLES
from pathlib import Path
import multiprocessing
import pandas as pd
import threading
import pypyodbc
import warnings
import sys

TIMEOUT_SECONDS = 3 * 60   # 3 minutes

### Code

In [2]:
# list of tables to export
tables = pd.read_excel(DATA_DIR / 'Tables_name.xlsx').iloc(axis=1)[0].tolist()

In [3]:
# remember to set up the connection in ODBC Data Source Administrator 64 bit
connection_string = (
    f"DSN={dsn};"
    f"UID={user};"
    f"PWD={passwd};"
)

In [4]:
def log_failed(table: str, reason: str):
    """Append failed table info to the log."""
    with FAILED_TABLES.open("a", encoding="utf8") as f:
        f.write(f"{table}  ---  {reason}\n")

def run_query_with_timeout(query: str, conn, timeout: int):
    """Run a query in a separate thread so it can timeout."""
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(pd.read_sql, query, conn, dtype="object")
        return future.result(timeout=timeout)

with open(BLACKLISTED_TABLES, 'r', encoding='utf8') as f:
    blacklist = [line.strip() for line in f if line.strip()]

with open(FAILED_TABLES, 'r', encoding='utf8') as f:
    already_failed = [line.split('  ---  ')[0] for line in f if line.strip()]

try:
    existing_files = set(f.stem for f in PARQUET_DIR.glob("*.parquet"))
    print(f"{len(existing_files)} .parquet files found.")
except FileNotFoundError:
    print("Nonexistent PARQUET_DIR, all files will be created.")
    existing_files = set()

104 .parquet files found.


In [ ]:
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

conn = None

try:
    conn = pypyodbc.connect(connection_string, autocommit=True)
    print("Database connection established.")

    for i, table in enumerate(tables, start=1):
        print(f"Table {table}: {i} of {len(tables)}")
        if table in existing_files:
            print(f"    Skipping table {i}, .parquet file already exists.")
            continue
        if table in already_failed:
            print(f"    Skipping previously failed table: {i}")
            continue
        if table in blacklist:
            print(f"    Skipping blacklisted table: {i}")
            continue
        try:  
            query = f"SELECT * FROM {table}"

            try:
                df = run_query_with_timeout(query, conn, TIMEOUT_SECONDS)
            except TimeoutError:
                print(f"    Timeout for {table}", file=sys.stderr)
                log_failed(table, "timeout")
                continue

            df = df.astype('string')  # convert all columns to string dtype to avoid type issues
            df = df.replace(['', ' ', 'NULL'], pd.NA) # replace empty strings and 'NULL' with NaN
            df.to_parquet(PARQUET_DIR / f"{table}.parquet", index=False)          

        except Exception as e:
            print(f"    An error occurred: {e}", file=sys.stderr)
            log_failed(table, str(e))
            continue

    print("Data export completed successfully.")

except pypyodbc.Error as e:
    print(f"Database connection error: {e}", file=sys.stderr)

except Exception as e:
    print(f"    An unexpected error occurred: {e}", file=sys.stderr)        

finally:
    if conn:
        conn.close()
        print("Database connection closed.")

Database connection established.
Table ADM_Bitacora_Conteos: 1 of 169
    Skipping table 1, .parquet file already exists.
Table ADM_Categorias: 2 of 169
    Skipping table 2, .parquet file already exists.
Table ADM_Conceptos: 3 of 169
    Skipping table 3, .parquet file already exists.
Table ADM_ConceptosEmpleados: 4 of 169
    Skipping table 4, .parquet file already exists.
Table ADM_ConfiguracionProrrateado: 5 of 169
    Skipping table 5, .parquet file already exists.
Table ADM_Conteos: 6 of 169
    Skipping table 6, .parquet file already exists.
Table ADM_EstatusConteos: 7 of 169
    Skipping table 7, .parquet file already exists.
Table ADM_GastosCortesY: 8 of 169
    Skipping table 8, .parquet file already exists.
Table ADM_IngresosEgresos: 9 of 169
    Skipping blacklisted table: 9
Table ADM_Prorrateos: 10 of 169
    Skipping previously failed table: 10
Table ADM_Prorrateos_Detalles: 11 of 169
    Skipping table 11, .parquet file already exists.
Table ADM_SobrantesFaltantes: 12 of